The problem is that D57, 59, 63, 65, 69, 71 are in one format, and are loaded into D_Data/GlobalLocal/trialData.csv.  
While the newer subjects are in a different format, within D_Data/GlobalLocal/D##/Trials.csv. And I need to combine these two into one csv.

C:\Users\jz421\Box\CoganLab\D_Data\GlobalLocal\D100\All blocks\GL_MainTask_Data_100_2023_10_21_10_30.csv is raw data

In [2]:
import pandas as pd
import os
import numpy as np

C:\Users\jz421\AppData\Local\Temp\ipykernel_8692\3363072451.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# Set the directory where the Excel files are located
rawDataFolder = r'C:\Users\jz421\Box\CoganLab\D_Data\GlobalLocal\rawDataCopies'

# Initialize an empty list to store DataFrames
dataframes_list = []

# Loop through all the files in the directory
for filename in os.listdir(rawDataFolder):
    # Check if the file is a csv
    if filename.endswith('.csv'):
        # Construct the full file path
        file_path = os.path.join(rawDataFolder, filename)
        # Read the Excel file into a pandas DataFrame
        df = pd.read_csv(file_path)
        # Append the DataFrame to the list
        dataframes_list.append(df)

# Combine all DataFrames in the list into a single DataFrame
combined_df = pd.concat(dataframes_list, ignore_index=True)

# Now, combined_df contains all the data from the Excel files in the folder


# Function to format the subject_ID column
def format_subject_id(subject_id):
    # Ensure the subject_id is a string
    subject_id_str = str(subject_id)
    # Check if it starts with 'D', if not, prepend 'D'
    if not subject_id_str.startswith('D'):
        subject_id_str = 'D' + subject_id_str
    # Split the 'D' from the number and zero-pad the number to 4 digits
    parts = subject_id_str.split('D')
    formatted_id = 'D' + parts[1].zfill(4)  # Zero-fill the numeric part
    return formatted_id

# Apply the function to the 'subject_ID' column
combined_df['subject_ID'] = combined_df['subject_ID'].apply(format_subject_id)

# Now df['subject_ID'] will have 'D' followed by a zero-padded four-digit number

In [7]:
combined_df['subject_ID']

KeyError: False

create numpy arrays that reflect each subject's accuracy on each trial, so i can later only analyze correct trials

In [6]:

# Get unique subjects
unique_subjects = combined_df['subject_ID'].unique()

# Directory to save the numpy arrays
save_dir = r'C:\Users\jz421\Box\CoganLab\D_Data\GlobalLocal\accArrays'  # Replace with your path
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Iterate over each subject and create a numpy array of accuracy
for subject in unique_subjects:
    # Filter DataFrame for the current subject
    subject_df = combined_df[combined_df['subject_ID'] == subject]
    # Sort the DataFrame based on trialCount to ensure order
    subject_df = subject_df.sort_values(by='trialCount')
    # Create numpy array from 'acc' column
    accuracy_array = subject_df['acc'].values
    # Save the numpy array to a file
    np.save(os.path.join(save_dir, f'{subject}_accuracy.npy'), accuracy_array)

# This will save a .npy file for each subject in the specified directory

In [25]:
combined_df.to_csv(r'C:\Users\jz421\Box\CoganLab\D_Data\GlobalLocal\combinedData.csv', index=False)